<a href="https://colab.research.google.com/github/alexandergribenchenko/EDVai_DE/blob/main/EDVai_F_Transformations/NB_PySpark_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A. Instalación y carga de Pyspark

In [2]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('test_pyspark').getOrCreate()

In [55]:
from pyspark.sql.functions import sum, col, desc, asc, count, countDistinct, round, max, min, avg, to_date, collect_set

# Librerías necesarias

In [37]:
dfs = spark.read.parquet('yellow_tripdata_2021-01.parquet')

In [13]:
dfs.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2021-01-01 00:30:10|  2021-01-01 00:36:12|            1.0|          2.1|       1.0|                 N|         142|          43|           2|        8.0|  3.0|    0.5|       0.

In [15]:
dfs.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



## 03.01.
Mostrar los resultados siguientes:
- a. VendorId Integer
- b. Tpep_pickup_datetime date
- c. Total_amount double
- d. Donde el total (total_amount sea menor a 10 dólares)

In [21]:
dfs.select('VendorID','tpep_pickup_datetime', 'total_amount').\
    filter(col('total_amount') < 10).\
    show()

+--------+--------------------+------------+
|VendorID|tpep_pickup_datetime|total_amount|
+--------+--------------------+------------+
|       1| 2021-01-01 00:51:20|         4.3|
|       2| 2021-01-01 00:42:11|         8.3|
|       2| 2021-01-01 00:04:21|        9.96|
|       2| 2021-01-01 00:43:41|         9.3|
|       2| 2021-01-01 00:36:08|         5.8|
|       1| 2021-01-01 00:03:13|         0.0|
|       1| 2021-01-01 00:30:32|         9.3|
|       2| 2021-01-01 00:16:19|         9.8|
|       2| 2021-01-01 00:57:26|         8.8|
|       2| 2021-01-01 00:33:33|        9.96|
|       2| 2021-01-01 00:39:08|         9.3|
|       2| 2021-01-01 00:45:29|         7.8|
|       1| 2021-01-01 00:43:00|        9.55|
|       2| 2021-01-01 00:52:28|         4.8|
|       1| 2021-01-01 00:39:50|         9.8|
|       2| 2021-01-01 00:17:39|         8.8|
|       1| 2021-01-01 00:49:10|         7.8|
|       2| 2021-01-01 00:47:51|        9.36|
|       2| 2021-01-01 00:11:53|         8.3|
|       1|

## 03.02.
Mostrar los 10 días que más se recaudó dinero (tpep_pickup_datetime, total amount)

In [39]:
dfs.withColumn('tpep_pickup_date', to_date('tpep_pickup_datetime', 'yyyy-MM-dd HH:mm:ss')).\
    groupBy('tpep_pickup_date').\
    agg(sum('total_amount').alias('sum_total_amount')).\
    orderBy(desc('sum_total_amount')).\
    limit(10).\
    show()

+----------------+-----------------+
|tpep_pickup_date| sum_total_amount|
+----------------+-----------------+
|      2021-01-28|959114.4900002397|
|      2021-01-22|933129.1800002002|
|      2021-01-29|929731.0600002115|
|      2021-01-21| 929307.270000204|
|      2021-01-14|925183.8200001806|
|      2021-01-15|924665.2000001943|
|      2021-01-27|894418.6400001668|
|      2021-01-19|889278.4600001582|
|      2021-01-07|886008.2300001475|
|      2021-01-13|873117.0800001248|
+----------------+-----------------+



## 03.03.
Mostrar los 10 viajes que menos dinero recaudó en viajes mayores a 10 millas

In [41]:
dfs.select('trip_distance','total_amount').\
    filter(col('trip_distance') > 10).\
    orderBy(asc('total_amount')).\
    show()

+-------------+------------+
|trip_distance|total_amount|
+-------------+------------+
|        12.68|      -252.3|
|        34.35|     -176.42|
|        14.75|      -152.8|
|        33.96|     -127.92|
|         29.1|      -119.3|
|        26.94|      -111.3|
|        20.08|      -107.8|
|        19.55|      -102.8|
|        19.16|      -90.55|
|        25.83|      -88.54|
|        32.85|       -85.3|
|        14.46|      -84.55|
|        23.63|       -80.8|
|         17.8|      -78.55|
|        24.44|      -78.04|
|        25.31|      -77.92|
|        15.79|      -77.05|
|        33.07|      -76.42|
|        15.14|      -75.55|
|        23.74|       -73.8|
+-------------+------------+
only showing top 20 rows



## 03.04.
Mostrar los viajes de más de dos pasajeros que hayan pagado con tarjeta de
crédito (mostrar solo las columnas trip_distance y tpep_pickup_datetime)

Sabemos que en `Payment_type` tenemos la equivalencia `1= Credit card`.

In [49]:
dfs.select('trip_distance','total_amount', 'payment_type', 'passenger_count').\
    filter((col('payment_type') == 1)&(col('passenger_count') > 2)).\
    show()

+-------------+------------+------------+---------------+
|trip_distance|total_amount|payment_type|passenger_count|
+-------------+------------+------------+---------------+
|         6.11|        24.3|           1|            3.0|
|          1.7|       14.16|           1|            5.0|
|         3.15|        18.3|           1|            4.0|
|        10.74|        40.3|           1|            3.0|
|         2.01|        14.8|           1|            5.0|
|         2.85|       18.59|           1|            3.0|
|         1.68|       13.56|           1|            3.0|
|         0.77|        9.96|           1|            3.0|
|          0.4|         7.8|           1|            3.0|
|        16.54|       66.36|           1|            3.0|
|          2.3|       15.95|           1|            3.0|
|          2.5|        15.8|           1|            3.0|
|         1.29|       11.76|           1|            3.0|
|          1.3|       12.95|           1|            3.0|
|          0.8

## 03.05.
Mostrar los 7 viajes con mayor propina en distancias mayores a 10 millas (mostrar campos tpep_pickup_datetime, trip_distance, passenger_count, ip_amount)

In [53]:
dfs.select('trip_distance','tpep_pickup_datetime','passenger_count', 'tip_amount').\
    filter(col('trip_distance') > 10).\
    orderBy(desc('tip_amount')).\
    limit(7).\
    show()

+-------------+--------------------+---------------+----------+
|trip_distance|tpep_pickup_datetime|passenger_count|tip_amount|
+-------------+--------------------+---------------+----------+
|        427.7| 2021-01-20 11:22:05|            1.0|   1140.44|
|        267.7| 2021-01-03 11:36:52|            1.0|     369.4|
|        326.1| 2021-01-12 12:57:36|            0.0|    192.61|
|        260.5| 2021-01-19 11:38:47|            1.0|    149.03|
|         11.1| 2021-01-31 23:48:50|            0.0|     100.0|
|        14.86| 2021-01-01 15:26:43|            2.0|      99.0|
|         13.0| 2021-01-18 15:50:24|            0.0|      90.0|
+-------------+--------------------+---------------+----------+



## 03.06.
Mostrar para cada uno de los valores de RateCodeID, el monto total y el monto promedio. Excluir los viajes en donde RateCodeID es ‘Group Ride’.

In [58]:
RatecodeID_uniques = sorted(dfs.select(collect_set('RatecodeID')).first()[0])
RatecodeID_uniques

[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 99.0]

Sabemos que en `RateCodeID` tenemos la equivalencia `6=Group ride`.

In [60]:
dfs.filter(col('RatecodeID') != 6).\
    groupBy('RatecodeID').\
    agg(sum('total_amount').alias('sum_total_amount'), avg('total_amount').alias('avg_total_amount') ).\
    show()

+----------+--------------------+------------------+
|RatecodeID|    sum_total_amount|  avg_total_amount|
+----------+--------------------+------------------+
|       1.0|1.9496468430212937E7|15.606626116946773|
|       4.0|   90039.93000000082| 74.90842762063296|
|       3.0|   67363.26000000043| 78.69539719626219|
|       2.0|   973635.4700000732| 65.52937609369182|
|      99.0|  1748.0699999999997| 48.55749999999999|
|       5.0|  255075.08999999086|48.939963545662096|
+----------+--------------------+------------------+

